In [1]:
# import modules
import panel as pn
pn.extension('tabulator')
import pandas as pd
import numpy as np
from panel.template import FastListTemplate
from pathlib import Path
#from yahoo_fin.stock_info import get_data
import datetime
from matplotlib.figure import Figure
from matplotlib import cm
%matplotlib inline
import hvplot.pandas
import holoviews as hv
from holoviews import opts


import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import modules that help build tabs
#import modules.helpers as helpers
#import modules.HistoricalData as hst
#import modules.MCTab as MCTab
#import modules.intro as intro
#import modules.profile as prf
#import modules.algorithmic_functions as af


#import pandas_ta as ta
#import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

#from pandas.tseries.offsets import DateOffset
#from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, f1_score
#import seaborn as sns

#from joblib import dump, load

2023-04-12 13:47:44.178190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Data

## *Data has been preprocessed*

* If necessary/desired, use 'build_portfolio_signal_ml_df' to pull machine learning data to create/refresht he test/train datasets
* Load the test/train datasets
* uncomment the below code in order to create or refresh the test/train datasets

In [2]:
# load X_train_full and X_test_full
X_train_full_conservative = pd.read_csv(Path("./data/X_train_full_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_conservative = pd.read_csv(Path("./data/X_test_full_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_balanced = pd.read_csv(Path("./data/X_train_full_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_balanced = pd.read_csv(Path("./data/X_test_full_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_growth = pd.read_csv(Path("./data/X_train_full_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_growth = pd.read_csv(Path("./data/X_test_full_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_aggressive = pd.read_csv(Path("./data/X_train_full_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_aggressive = pd.read_csv(Path("./data/X_test_full_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_full_alternative = pd.read_csv(Path("./data/X_train_full_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_full_alternative = pd.read_csv(Path("./data/X_test_full_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)



#load y_train and y_test
y_train_conservative = pd.read_csv(Path("./data/y_train_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_conservative = pd.read_csv(Path("./data/y_test_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_balanced = pd.read_csv(Path("./data/y_train_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_balanced = pd.read_csv(Path("./data/y_test_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_growth = pd.read_csv(Path("./data/y_train_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_growth = pd.read_csv(Path("./data/y_test_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_aggressive = pd.read_csv(Path("./data/y_train_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_aggressive = pd.read_csv(Path("./data/y_test_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_alternative = pd.read_csv(Path("./data/y_train_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_alternative = pd.read_csv(Path("./data/y_test_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

datafiles = {'conservative': [X_train_full_conservative,
                              X_test_full_conservative, 
                              y_train_conservative, 
                              y_test_conservative],
            'balanced': [X_train_full_balanced,
                              X_test_full_balanced, 
                              y_train_balanced, 
                              y_test_balanced],
            'growth': [X_train_full_growth,
                              X_test_full_growth, 
                              y_train_growth, 
                              y_test_growth],
            'aggressive': [X_train_full_aggressive,
                              X_test_full_aggressive, 
                              y_train_aggressive, 
                              y_test_aggressive],
            'alternative': [X_train_full_alternative,
                              X_test_full_alternative, 
                              y_train_alternative, 
                              y_test_alternative]}

portfolios = ['conservative', 'balanced', 'growth', 'aggressive','alternative']

# Create Model

In [3]:
# Create the scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train_full_conservative)

# Scale the data
X_train_full_conservative_scaled = X_scaler.transform(X_train_full_conservative)
X_test_full_conservative_scaled = X_scaler.transform(X_test_full_conservative)




In [4]:

results_full = pd.DataFrame()
X_scaler = StandardScaler()
for p in portfolios:
    X_train = datafiles[p][0]
    X_test = datafiles[p][1]
    y_train = datafiles[p][2]
    y_test = datafiles[p][3]
    
    scaler = X_scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    number_input_features = 18
    def model_builder(hp):

        hp_units = hp.Int('units', min_value=6, max_value=36, step=2)
        hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
        hp_units2 = hp.Int('units2', min_value=3, max_value=18, step=2)
        hp_activation2 = hp.Choice('activation2', values=['relu', 'tanh'])
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model = Sequential()
        model.add(Dense(units=hp_units, input_dim=number_input_features, activation=hp_activation))
        model.add(Dense(units=hp_units2, activation=hp_activation2))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])

        return model

    tuner = kt.Hyperband(model_builder, objective='val_accuracy',
                 max_epochs=100,
                 factor=3,
                 directory='./hp',
                 project_name=f"hp_{p}")

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    print(f"now running tuner search for {p}")
    tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2, callbacks=[stop_early], verbose=0)
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_unit1 = best_hps.get('units')
    best_unit2 = best_hps.get('units2') 
    best_activation1 =  best_hps.get('activation')
    best_activation2 = best_hps.get('activation2')
    best_learning_rate = best_hps.get('learning_rate')

    best_dict = {'units1': best_unit1,
                 'units2': best_unit2,
                 'activation1': best_activation1,
                 'activation2': best_activation2,
                 'learning_rate': best_learning_rate}

    best_parameters = pd.DataFrame.from_dict(best_dict, orient='index', columns=[f"{p}"])
    results_full = pd.concat([results_full, best_parameters], axis=1)
        

2023-04-12 13:47:54.050620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


now running tuner search for conservative
INFO:tensorflow:Oracle triggered exit
now running tuner search for balanced
INFO:tensorflow:Oracle triggered exit
now running tuner search for growth
INFO:tensorflow:Oracle triggered exit
now running tuner search for aggressive
INFO:tensorflow:Oracle triggered exit
now running tuner search for alternative
INFO:tensorflow:Oracle triggered exit


In [5]:
results_full

,conservative,balanced,growth,aggressive,alternative
units1,8,30,14,8,28
units2,3,5,5,9,3
activation1,relu,tanh,tanh,tanh,tanh
activation2,tanh,relu,tanh,tanh,relu
learning_rate,0.0001,0.01,0.001,0.0001,0.01


In [6]:
# load X_train_reduced and X_test_reduced
X_train_reduced_conservative = pd.read_csv(Path("./data/X_train_reduced_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_conservative = pd.read_csv(Path("./data/X_test_reduced_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_balanced = pd.read_csv(Path("./data/X_train_reduced_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_balanced = pd.read_csv(Path("./data/X_test_reduced_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_growth = pd.read_csv(Path("./data/X_train_reduced_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_growth = pd.read_csv(Path("./data/X_test_reduced_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_aggressive = pd.read_csv(Path("./data/X_train_reduced_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_aggressive = pd.read_csv(Path("./data/X_test_reduced_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

X_train_reduced_alternative = pd.read_csv(Path("./data/X_train_reduced_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
X_test_reduced_alternative = pd.read_csv(Path("./data/X_test_reduced_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)



#load y_train and y_test
y_train_conservative = pd.read_csv(Path("./data/y_train_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_conservative = pd.read_csv(Path("./data/y_test_conservative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_balanced = pd.read_csv(Path("./data/y_train_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_balanced = pd.read_csv(Path("./data/y_test_balanced.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_growth = pd.read_csv(Path("./data/y_train_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_growth = pd.read_csv(Path("./data/y_test_growth.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_aggressive = pd.read_csv(Path("./data/y_train_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_aggressive = pd.read_csv(Path("./data/y_test_aggressive.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

y_train_alternative = pd.read_csv(Path("./data/y_train_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)
y_test_alternative = pd.read_csv(Path("./data/y_test_alternative.csv"), index_col="Unnamed: 0", parse_dates=True, infer_datetime_format=True)

datafiles = {'conservative': [X_train_reduced_conservative,
                              X_test_reduced_conservative, 
                              y_train_conservative, 
                              y_test_conservative],
            'balanced': [X_train_reduced_balanced,
                              X_test_reduced_balanced, 
                              y_train_balanced, 
                              y_test_balanced],
            'growth': [X_train_reduced_growth,
                              X_test_reduced_growth, 
                              y_train_growth, 
                              y_test_growth],
            'aggressive': [X_train_reduced_aggressive,
                              X_test_reduced_aggressive, 
                              y_train_aggressive, 
                              y_test_aggressive],
            'alternative': [X_train_reduced_alternative,
                              X_test_reduced_alternative, 
                              y_train_alternative, 
                              y_test_alternative]}

portfolios = ['conservative', 'balanced', 'growth', 'aggressive','alternative']

In [7]:
X_scaler = StandardScaler()
results_reduced = pd.DataFrame()
for p in portfolios:
    X_train = datafiles[p][0]
    X_test = datafiles[p][1]
    y_train = datafiles[p][2]
    y_test = datafiles[p][3]
    
    scaler = X_scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    number_input_features = 7
    def model_builder(hp):

        hp_units = hp.Int('units', min_value=6, max_value=36, step=2)
        hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
        hp_units2 = hp.Int('units2', min_value=3, max_value=18, step=2)
        hp_activation2 = hp.Choice('activation2', values=['relu', 'tanh'])
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model = Sequential()
        model.add(Dense(units=hp_units, input_dim=number_input_features, activation=hp_activation))
        model.add(Dense(units=hp_units2, activation=hp_activation2))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])

        return model

    tuner = kt.Hyperband(model_builder, objective='val_accuracy',
                 max_epochs=100,
                 factor=3,
                 directory='./hp',
                 project_name=f"hp__reduced_{p}")

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    print(f"now running tuner search for {p}")
    tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2, callbacks=[stop_early], verbose=0)
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_unit1 = best_hps.get('units')
    best_unit2 = best_hps.get('units2') 
    best_activation1 =  best_hps.get('activation')
    best_activation2 = best_hps.get('activation2')
    best_learning_rate = best_hps.get('learning_rate')

    best_dict = {'units1': best_unit1,
                 'units2': best_unit2,
                 'activation1': best_activation1,
                 'activation2': best_activation2,
                 'learning_rate': best_learning_rate}
     
    print(f"best values for {p} are:")
    for k,v in best_dict.items():
        print(k, v)

    best_parameters = pd.DataFrame.from_dict(best_dict, orient='index', columns=[f"{p}"])
    results_reduced = pd.concat([results_reduced, best_parameters], axis=1)
        

now running tuner search for conservative
INFO:tensorflow:Oracle triggered exit
best values for conservative are:
units1 36
units2 3
activation1 tanh
activation2 tanh
learning_rate 0.01
now running tuner search for balanced
INFO:tensorflow:Oracle triggered exit
best values for balanced are:
units1 14
units2 7
activation1 relu
activation2 relu
learning_rate 0.001
now running tuner search for growth
INFO:tensorflow:Oracle triggered exit
best values for growth are:
units1 20
units2 3
activation1 relu
activation2 tanh
learning_rate 0.01
now running tuner search for aggressive
INFO:tensorflow:Oracle triggered exit
best values for aggressive are:
units1 34
units2 7
activation1 tanh
activation2 relu
learning_rate 0.01
now running tuner search for alternative
INFO:tensorflow:Oracle triggered exit
best values for alternative are:
units1 18
units2 7
activation1 relu
activation2 tanh
learning_rate 0.001


In [8]:
results_reduced

,conservative,balanced,growth,aggressive,alternative
units1,36,14,20,34,18
units2,3,7,3,7,7
activation1,tanh,relu,relu,tanh,relu
activation2,tanh,relu,tanh,relu,tanh
learning_rate,0.01,0.001,0.01,0.01,0.001


In [9]:
results_full

,conservative,balanced,growth,aggressive,alternative
units1,8,30,14,8,28
units2,3,5,5,9,3
activation1,relu,tanh,tanh,tanh,tanh
activation2,tanh,relu,tanh,tanh,relu
learning_rate,0.0001,0.01,0.001,0.0001,0.01
